In [1]:
from plyfile import PlyData
import numpy as np
import curvature as cur

import utilities as uti
import open3d as o3d

def initData(plydata):
    cnt = plydata['vertex'].count
    mesh = np.empty((cnt,5))
    mesh[:,0] = plydata['vertex']['x']
    mesh[:,1] = plydata['vertex']['y']
    mesh[:,2] = plydata['vertex']['z']
    mesh[:,3] = plydata['vertex']['quality']
    mesh[:,4] = plydata['vertex']['nz']
    return mesh

for i in range(2,3):
    ## read data
    plydata = PlyData.read('./Mesh/results/8 frames/jean_ng'+str(i)+'.ply')
    
    ## init
    mesh = initData(plydata)

    ## get peak areas curvature
    mesh = mesh[np.where(mesh[:,4] < 0)]
    high = mesh[np.where(mesh[:,3] > 50)]
    low = mesh[np.where(mesh[:,3] < -70)]

    ## Concave joints low gaussian curvature
    concave = uti.createPointCloud(low[:,:3])
    #concave = uti.outlierRemoval(concave,10,0.5)['inlier']

    ## Convex joints high gaussian curvature
    convex = uti.createPointCloud(high[:,:3])
    convex = uti.outlierRemoval(convex,10,0.1)['inlier']



    # visualization
    # concave.paint_uniform_color(np.array([0,0,1]))
    # convex.paint_uniform_color(np.array([1,0,0]))
    # o3d.visualization.draw_geometries([concave,convex])

In [2]:
from human_skeleton import HumanSkeleton

convexs = np.asarray(convex.points)
concaves = np.asarray(concave.points)
maxi = np.amax(convexs[:,1])
mini = np.amin(convexs[:,1])
hei = maxi - mini

def getJointPoint(points,limit_top,limit_bot,num=1,note=False):
    data = np.array([x for x in points if limit_bot<=x[1]<=limit_top])
    if num == 1:
        temp = []
        for i in data:
            temp.append(i)
        temp = np.asarray(temp)
        return np.array([np.array([np.average(temp[:,0]),np.average(temp[:,1]),np.average(temp[:,2])])])
    elif num == 2:
        left = []
        right = []
        center = {
            'x' : (np.amax(points[:,0]) + np.amin(points[:,0]))/2,
            'y' : (np.amax(points[:,1]) + np.amin(points[:,1]))/2,
            'z' : (np.amax(points[:,2]) + np.amin(points[:,2]))/2,
        }
        for i in data:
            if i[0] < center['x']:
                left.append(i)
            else:
                right.append(i)
        left = np.asarray(left)
        right = np.asarray(right)
        result = []
        try:
            result.append([np.average(left[:,0]),np.average(left[:,1]),np.average(left[:,2])])
        except:
            if note:
                temp = note + ' '
            else:
                temp = ''
            print('Cant get '+temp+'left point!')
        try:
            result.append([np.average(right[:,0]),np.average(right[:,1]),np.average(right[:,2])])
        except:
            if note:
                temp = note + ' '
            else:
                temp = ''
            print('Cant get '+temp+'right point!')
        return np.array(result)
    else:
        print('Only have 1 or 2 joint(s)!')
        return False

## convex
head = getJointPoint(convexs, (mini + hei), (mini + 0.87*hei))
shoulders = getJointPoint(convexs, (mini + 0.869*hei), (mini + 0.8*hei),2)
hands = getJointPoint(convexs, (mini + 0.6*hei), (mini + 0.4*hei),2)
knees = getJointPoint(convexs, (mini + 0.37*hei), (mini + 0.3*hei),2)
foots = getJointPoint(convexs, (mini + 0.2*hei), (mini + 0.01*hei),2)

## concave
neck = getJointPoint(concaves, (mini + hei), (mini + 0.8*hei))
elbows = getJointPoint(concaves, (mini + 0.7*hei), (mini + 0.6*hei),2)
wrists = getJointPoint(concaves, (mini + 0.6*hei), (mini + 0.5*hei),2)
weist = getJointPoint(concaves, (mini + 0.45*hei), (mini + 0.42*hei))
ankles = getJointPoint(concaves, (mini + 0.2*hei), (mini + 0.05*hei),2)

joints = np.concatenate((head,neck,shoulders,elbows,wrists,hands,weist,knees,ankles,foots))

joints_pcl = uti.createPointCloud(joints)
joints_pcl.paint_uniform_color(np.array([1,0,0]))

ori = uti.createPointCloud(mesh[:,:3])
ori.paint_uniform_color(np.array([0.8,0.8,0.8]))

#o3d.visualization.draw_geometries([joints])

geometry::PointCloud with 1830 points.

In [3]:
jean = HumanSkeleton()
skeleton = jean.createSkeleton(joints)
colors = np.array([[1, 0, 0] for i in range(len(joints))])
skeleton.colors = o3d.utility.Vector3dVector(colors)
#o3d.visualization.draw_geometries([skeleton,ori,joints_pcl])

### Skeleton from Kinect

In [4]:
import data_to_plist as dtp
kinect_skeleton = dtp.generatePoints('./Skeletons Kinect/skt2.dat')
kinect_skeleton = uti.createPointCloud(kinect_skeleton)
kinect_skeleton.paint_uniform_color(np.array([0,1,0]))

geometry::PointCloud with 25 points.

In [5]:
o3d.visualization.draw_geometries([skeleton,joints_pcl,kinect_skeleton])

In [7]:
convex = np.concatenate((head,shoulders,hands,knees,foots))
concave = np.concatenate((neck,elbows,wrists,weist,ankles))
convex = uti.createPointCloud(convex)
concave = uti.createPointCloud(concave)
convex.paint_uniform_color(np.array([1,0,0]))
concave.paint_uniform_color(np.array([0,1,0]))
o3d.visualization.draw_geometries([concave,convex,ori])

In [ ]:
pcd = o3d.io.read_point_cloud('./Mesh/results/8 frames/jean_ng2.ply')